In [34]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [35]:
class BatsmanState(TypedDict):
    balls: int
    score: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_perc: float
    summary:str

In [36]:
def StrikeRate(state: BatsmanState) -> BatsmanState:
    sr= round((state['score']/ state['balls'])*100,2)
    return {'sr':sr}

In [37]:
def calculate_bpb(state: BatsmanState) -> BatsmanState:
    bpb= round(state['score']/(state['fours']+state['sixes']),2)

    return {'bpb':bpb}


In [38]:
def boundary_percentage(state: BatsmanState) -> BatsmanState:
    bp= round((((state['fours']*4) + (state['sixes']*6))/state['score'])*100,2)

    return {'boundary_perc':bp}

In [39]:
def summary(state: BatsmanState) -> BatsmanState:
    summary= f"""
    Strike Rate - {state['sr']}\n
    Bowls Per Boundary - {state['bpb']}\n
    Boundary Percentage - {state['boundary_perc']}.
"""
    return {'summary': summary}

In [40]:
# Create Graph
graph= StateGraph(BatsmanState)

# Add Node
graph.add_node('Strike Rate', StrikeRate)
graph.add_node('calculate bpb', calculate_bpb)
graph.add_node('boundary percentage', boundary_percentage)
graph.add_node('summary', summary)

# Add Edges
graph.add_edge(START,'Strike Rate')
graph.add_edge(START,'calculate bpb')
graph.add_edge(START, 'boundary percentage')

graph.add_edge('Strike Rate', 'summary')
graph.add_edge('calculate bpb', 'summary')
graph.add_edge('boundary percentage', 'summary')

graph.add_edge('summary', END)

workflow= graph.compile()

In [41]:
print(workflow.get_graph().draw_ascii())

                                +-----------+                                 
                               *| __start__ |*                                
                          ***** +-----------+ *****                           
                     *****            *            *****                      
                *****                 *                 *****                 
             ***                      *                      ***              
+-------------+           +---------------------+           +---------------+ 
| Strike Rate |*          | boundary percentage |           | calculate bpb | 
+-------------+ *****     +---------------------+       ****+---------------+ 
                     *****            *            *****                      
                          *****       *       *****                           
                               ***    *    ***                                
                                +---------+         

In [43]:
initial_state= {
    "balls": 91,
    "score": 154,
    "fours": 4,
    "sixes": 9,
}
print(workflow.invoke(initial_state))

{'balls': 91, 'score': 154, 'fours': 4, 'sixes': 9, 'sr': 169.23, 'bpb': 11.85, 'boundary_perc': 45.45, 'summary': '\n    Strike Rate - 169.23\n\n    Bowls Per Boundary - 11.85\n\n    Boundary Percentage - 45.45.\n'}
